In [33]:
import pandas as pd
import numpy as np
import xarray as xr
import os
from warnings import filterwarnings

In [34]:
# Lendo arquivos netCDF
diretorio = r'C:\IC\netCDF'

# Listando arquivos
arquivos = sorted([os.path.join(diretorio, arquivo) for arquivo in os.listdir(diretorio) if arquivo.endswith('.nc')])

In [35]:
# Abrindo arquivo de coordenadas
pontos = pd.read_csv(r'C:\IC\Scripts2\pontos.csv', sep=',')

# listando coordenadas
latitudes = pontos['lat'].values
longitudes = pontos['lon'].values

In [36]:
# Corrigindo longitude
for arquivo in arquivos:
    # Abrindo arquivo netCDF
    netCDF = xr.open_dataset(arquivo)

    # Filtrando coordenadas
    longitude_corrigida = netCDF['longitude'].values

    # Corrigindo longitude
    longitude_corrigida[longitude_corrigida > 180] = longitude_corrigida[longitude_corrigida > 180] - 360

    # Salvando coordenadas corrigidas
    netCDF.to_netcdf(arquivo.replace('.nc', '_corrigido.nc'))

In [37]:
# diretório dos arquivos corrigidos
diretorio_corrigido = r'C:\IC\netCDF\corrigido'

# Listando arquivos
arquivos_corrigido = sorted([os.path.join(diretorio_corrigido, arquivo) for arquivo in os.listdir(diretorio_corrigido) if arquivo.endswith('.nc')])


In [38]:
# criando dataframe
dataset = pd.DataFrame(columns=['data','horario', 'latitude', 'longitude', 'chuva'])

# laço que percorre todos os arquivos e relaciona com os pontos no arquivo csv
for arquivo in arquivos_corrigido:
    
    # pegando a data e horário do nome do arquivo
    data,horario = arquivo[35:43], arquivo[43:45]

    # Abrindo arquivo netCDF
    netCDF = xr.open_dataset(arquivo)

    # Acessando variável de interesse
    netCDF = netCDF['prec']

    # laço que percorre todos os pontos
    for i in range(len(pontos)):

        # Pegando o valor de chuva em um ponto
        chuva = netCDF.sel(latitude=pontos.iloc[i].values[0], longitude=pontos.iloc[i].values[1], method='nearest').values
        
        # preenchendo o dataframe com linhas
        linha = [data, horario, pontos.iloc[i].values[0], pontos.iloc[i].values[1], chuva]

        # adicionando na ultima linha ao dataframe
        dataset.loc[len(dataset)] = linha


# Teste futuro para padronização das Datas

In [39]:
# Corrigindo valores de data
dataset['data'] = pd.to_datetime(dataset['data'], format='%Y%m%d')

# transformando datetime para string
dataset['data'] = dataset['data'].dt.strftime('%Y-%m-%d')

In [ ]:
# Buscando a coluna horário e substituindo os valores de 0 para 24
dataset.loc[dataset['horario'] == 0, 'horario'] = 24

# Datas para padronizar 
datas = ['2016-01-17', '2020-12-24', '2021-10-29']

# Buscando a coluna data e substituindo os dias 
# dataset.loc[dataset['data'].isin(datas), 'datas'] = '2016-01-01 00:00:00'

# Ajuste fino

In [ ]:
# pegando os dados únicos de chuva
datas = merge['data'].unique()

# transformando chuva horária para acumulado
for data in datas:
    for _,row in pontos.iterrows():
        lat = row[0]
        lon = row[1]
        merge.loc[(merge['data'] == data) & (merge['latitude'] == lat) & (merge['longitude'] == lon), 'chuva'] = merge.loc[(merge['data'] == data) & (merge['latitude'] == lat) & (merge['longitude'] == lon), 'chuva'].cumsum() 

In [41]:
# salvando o dataframe em um arquivo csv
dataset.to_csv(r'C:\IC\Scripts2\dataset-merge.csv', sep=',', index=False)